# Lab 1

<b>Class:</b> MSDS 7331 Data Mining
<br> <b>Dataset:</b> Belk Endowment Educational Attainment Data 

<h1 style="font-size:150%;"> Teammates </h1>
Maryam Shahini
<br> Murtada Shubbar
<br> Michael Toolin
<br> Steven Millett

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
import math
import re

%matplotlib inline

In [ ]:
#
# The 2017 Public Schools Machine Learning Date Set is being used throughout this analysis.  The _ML suffix is removed to less name space size
#
# Load Full Public School Data Frames for each year
#
school_data = pd.read_csv('./Data/2017/machine Learning Datasets/PublicSchools2017_ML.csv', low_memory=False)

# Business Understanding 

The North Carolina General Assmebly passed legislation in 2014-2014 requiring the assignment of School Performance Grades (SPG) for public and charter Schools [1].  This data set is collected in response to this legislation.  A school's SPG is calculated using 80% of the schools achievment score and 20% of the schools growth score.  The achievment score is calculated through a variety of student testing and the growth score is calculated using the EVASS School Accountablityy Growth Composite Index [2]. Schools are assigned a letter grade where A: 100-85 points, B: 84-70 points, C: 69-55 points, D: 54-40 points and F: less than 40 points.  Schools that receive grades of D or F are required by to inform parents of the school district.  In 2016, the North Carolina General Assmebly passed legislation creating the Achievment School District(ASD). This school district is run by a private organization and are run as charter schools [3].

This data set contains 334 features describing 2443 schools.  The data includes testing results used to derive the SPG described above.  It also contains school financial data, demographic information, attendence, and student behavior data measured by metrics such as susupension and expulsions. We can look into all these different types of information to see if any correlation with school performances exists, both good and bad.  Do poorly performing schools line up with any specific demographics?  Are there school financial situations that help attribute to a schools performance? Finding correlations of this data with SPG and being able to use that information in a predictive analysis algorithm may help educators identify schools before the perfomance metrics deteriorate, allowing them to intervene. The end result of all the testing and analaysis is providing all students a fair and equal opportunity at a qualtiy eduction.

[1] source: http://schools.cms.k12.nc.us/jhgunnES/Documents/School%20Performance%20Grade%20PP%20January%2014,%202015%20(1).pptx
[2] (EVASS Growth information available at http://www.ncpublicschools.org/effectiveness-model/evaas/selection/)
[3] source: https://www.ncforum.org/committee-on-low-performing-schools/

###citation: Drew J., The Belk Endowment Educational Attainment Data Repository for North Carolina Public Schools, (2018), GitHub repository, https://github.com/jakemdrew/EducationDataNC

Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). Describe how you would define and measure the outcomes from the dataset. That is, why is this data important and how do you know if you have mined useful knowledge from the dataset? How would you measure the effectiveness of a good prediction algorithm? Be specific.

# Data Meaning Type 

Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file.

The comprehensive description of all 334 attributes can be found in the data-dictionary.pdf associated with the NC Report Card database provided by Dr. Drew. We were interested in 60 variables moving forward in the course. We visualize several attributes of interest in this report.  

<img src="files/data_meaning.jpg">

In [ ]:
school_data.info(verbose=True)

In [ ]:
#scatter_matrix(school_data)

# Data Quality

Verify data quality: Explain any missing values, duplicate data, and outliers. Are those mistakes? How do you deal with these problems? Give justifications for your methods.
There is no missing values in our dataset to worry about.

In [ ]:
# to find the missing values if any:
#Print out all the missing value rows
pd.set_option('display.max_rows', 10000)

print('\r\n**The Remaining Missing Values Below will be set to Zero!**')

#Check for Missing values 
missing_values = school_data.isnull().sum().reset_index()
missing_values.columns = ['Variable Name', 'Number Missing Values']
missing_values = missing_values[missing_values['Number Missing Values'] > 0] 
missing_values


# Simple Statistics

Visualize appropriate statistics (e.g., range, mode, mean, median, variance, counts) for a subset of attributes. Describe anything meaningful you found from this or if you found something potentially interesting. Note: You can also use data from other sources for comparison. Explain why the statistics run are meaningful. 

In [ ]:
school_data.describe().transpose()

In [ ]:
school_data_simple = school_data[['SPG Score','lea_avg_student_num','student_num']]
school_data_simple.describe()

### Simple Statistics on SPG Score, and number of students in schools
The table above show us statistics for SPG.  Understanding the mean SPG score is 60.91, which is in the lower part of the "C" range.  The median SPG score
is 64.00, also in "C" range.  It is surprising to see a maximum SPG score of 100 as this indicates perfect scoring for some individual school.  Also the minimum score of 0 mean total failure in testing.  Both of these data points require more investigation to see if see if there are any errors in the data and where these points come from.

In [ ]:
# Find the schools whose SPG =1 00
school_SPG_100 =(school_data.loc[school_data['SPG Score'] == 100])
print (school_SPG_100['unit_code'])

The school with unit code 190501 has a perfect SPG. The data in this field should be examined for errors or to understand why they received a perfect SPG.

In [ ]:
# Find the schools whose SPG = 0  Purposely coded different than above to show various coding techniques
school_SPG_0 =(school_data['unit_code'].loc[school_data['SPG Score'] == 0])
print (school_SPG_0)

The list above shows 130 schools with SPG = 0.  This is more problematic than the single school whose SPG = 100.  Nonetheless we need to understand why 130 schools had an SPG = 0 and requires future investigation.

In [ ]:
school_data_finance = school_data[['lea_total_expense_num','lea_salary_expense_pct',
                                  'lea_services_expense_pct', 'lea_supplies_expense_pct',
                                  'lea_instruct_equip_exp_pct']]
school_data_finance.describe()

Looking at the expenses, the LEA with largest number of expenses is 90% higher than the mean for all LEA's in the state.  All other quantiles are closer to the mean, indicating a good chance this in outlier.  We will look for this school to see if there are errors in the data and consider how to handle this.

In [ ]:
demog_male_cols = [col for col in school_data.columns if "Male" in col]
demog_female_cols = [col for col in school_data.columns if "male" in col]
school_data_demog_male = school_data[np.intersect1d(school_data.columns, demog_male_cols)]
school_data_demog_female = school_data[np.intersect1d(school_data.columns, demog_female_cols)]
school_data_demog_all = pd.concat([school_data_demog_female, school_data_demog_male], axis=1)
school_data_demog = school_data_demog_all.filter(regex='^(?!(EOC|Two).*?)') 

school_data_demog.describe()

The above table shows actual quantile breakdown by minority demographics.  It is interesting to note there is at least one LEA that has 0 minorities and at least one where the entire male population is made up of minorities.  Identifying these LEA's may provide some information.

# Visualize Attributes

Visualize the most interesting attributes (at least 5 attributes, your opinion on what is interesting). Important: Interpret the implications for each visualization. Explain for each attribute why the chosen visualization is appropriate.

In [ ]:
def prefixsearch(search_string, missing_value ,start_of_search_string,end_of_search_string):
    if re.search(end_of_search_string, search_string):
        return re.search('('+ start_of_search_string +'\S*)(?='+ end_of_search_string +')',search_string).group(0)
    else:
        return missing_value



In [ ]:
teacher_temp_col = [col for col in school_data.columns if 'tchyrs' in col]
teacher_columns = school_data[teacher_temp_col].melt(var_name='col',value_name='values')

teacher_columns['year'] = teacher_columns['col'].apply(lambda name: re.search('(?<=tchyrs_)\S*(?=_)',name).group(0))
teacher_columns['region'] = teacher_columns['col'].apply(lambda name: prefixsearch(name, "Sch", '^\S*','_tch'))



In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(ax=ax,data=teacher_columns,x='year', y='values', hue='region');

# Makeup of teachers in classrooms
In the graph above we are looking at the percentage of teachers that makeup a classroom based on their years of tenure.

In [ ]:
sex_temp_col = school_data.filter(regex=('[Mm]alePct')).columns
sex_teacher_columns = school_data[sex_temp_col].melt(var_name='col',value_name='Values')

sex_teacher_columns['Race'] = sex_teacher_columns['col'].apply(lambda name: prefixsearch(name, "", '^','Male|Female'))
sex_teacher_columns['Sex'] = sex_teacher_columns['col'].apply(lambda name: 'Female' if re.search('Female',name) else 'Male')



In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(ax=ax,x='Race',y='Values',hue='Sex',data=sex_teacher_columns,palette={"Male": "b", "Female": "y"});

leg = plt.legend( loc = 'upper right')

plt.draw() # Draw the figure so you can find the positon of the legend. 

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change to location of the legend. 
xOffset = .1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)


# Update the plot
plt.show()

# Makeup of minorities in classrooms

In the graph above we are looking at the percentage of minorities that makeup a classroom as represented with a violin plot. Each half of the violin represents the different sexual make-up of each race.


In [ ]:
support_col = school_data.filter(regex=('wap|books|stud_internet')).columns
support_columns = school_data[support_col].melt(var_name='col',value_name='Values')

support_columns = support_columns[support_columns['Values']!=0]

support_columns['Values'] = support_columns['Values'].apply(lambda value: math.log(value))


#support_columns['media'] = sex_teacher_columns['col'].apply(lambda name: 'Female' if re.search('Female',name) else 'Male')

support_columns['region'] = support_columns['col'].apply(lambda name: prefixsearch(name, "Sch", '^\S*','_wap|_books|_stud_int'))
support_columns['media'] = support_columns['col'].apply(lambda name: re.sub('lea_','',name) if re.search('lea',name) else name )
support_columns = support_columns.sort_values(by='Values')
#print(support_columns.sample(20))

In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(ax=ax,x='media',y='Values',hue='region',data=support_columns,palette={"lea": "b", "Sch": "y"});
plt.xticks(rotation=30)

leg = plt.legend( loc = 'upper right')

plt.draw() # Draw the figure so you can find the positon of the legend. 

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change to location of the legend. 
xOffset = .1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)


# Update the plot
plt.show()

# Media and Computer resources available
These box plots map the difference in the amount of resources that the average LEA has and the variance per school.

In [ ]:
TCHR_col = school_data.filter(regex=('TCHR_Standard')).columns
TCHR_columns = school_data[TCHR_col].melt(var_name='col',value_name='Values')


TCHR_columns['Standard'] = TCHR_columns['col'].apply(lambda name: re.search('(?<=TCHR_).*(?=_Pct)',name).group(0))
TCHR_columns['Level'] = TCHR_columns['col'].apply(lambda name: re.search('(?<=^).*(?=_TCHR)',name).group(0))

print(TCHR_columns)

In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(ax=ax,x='Level',y='Values',hue='Standard',data=TCHR_columns);
plt.xticks(rotation=30)


leg = plt.legend( loc = 'upper right')

plt.draw() # Draw the figure so you can find the positon of the legend. 

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change to location of the legend. 
xOffset = .15
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)


# Update the plot
plt.show()

# Here is some text about the graph

In [ ]:
EOG_col = school_data.filter(regex=('EOGG')).columns
EOG_columns = school_data[EOG_col].melt(var_name='col',value_name='Values')

EOG_columns['Grade'] = EOG_columns['col'].apply(lambda name: re.search('(?<=EOG).*(?=_[CG])',name).group(0))
EOG_columns['Level'] = EOG_columns['col'].apply(lambda name: re.split('EOGGr[3-5]_',name)[1])

EOG_columns = EOG_columns.sort_values(by='Level')

In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
plot = sns.violinplot(ax=ax,x='Level',y='Values',hue='Grade',data=EOG_columns);
plt.xticks(rotation=30)

leg = plt.legend( loc = 'upper right',title='Grade')

plt.draw() # Draw the figure so you can find the positon of the legend. 

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change to location of the legend. 
xOffset = .1
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)


# Update the plot
plt.show()

# Here is some text about the EOG stats

In [ ]:
EOC_col = school_data.filter(regex=('^EOC')).columns
EOC_columns = school_data[EOC_col].melt(var_name='col',value_name='Values')

EOC_columns['Subject'] = EOC_columns['col'].apply(lambda name: re.search('(?<=EOC).*(?=_[CG])',name).group(0))
EOC_columns['Level'] = EOC_columns['col'].apply(lambda name: re.split('EOC(Subjects|MathI)_',name)[2])

EOC_columns = EOC_columns.sort_values(by='Level')

In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
sns.violinplot(ax=ax,x='Level',y='Values',hue='Subject',data=EOC_columns);
plt.xticks(rotation=30)

leg = plt.legend( loc = 'upper right',title='Subject')

plt.draw() # Draw the figure so you can find the positon of the legend. 

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change to location of the legend. 
xOffset = .12
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)


# Update the plot
plt.show()

# Here is some text about the EOG stats

# Explore Joint Attributes

Visualize relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain any interesting relationships.

Correlation matrix heatmap: between funding and SPG scores/benchmark grades ---add description.  

In [ ]:
first_heat = school_data[['lea_total_expense_num',
'lea_salary_expense_pct',
'lea_services_expense_pct',
'lea_supplies_expense_pct',
'lea_instruct_equip_exp_pct',
'lea_federal_perpupil_num',
'lea_local_perpupil_num',
'lea_state_perpupil_num',            
'SPG Score',
'Reading SPG Score',
'EVAAS Growth Score',
'lea_sat_avg_score_num',
'lea_sat_participation_pct',
'lea_ap_participation_pct',
'lea_ap_pct_3_or_above',]] 
                       
first_corr = first_heat.corr()

plt.figure(figsize = (14,10)) #size of matrix 
sns.heatmap(first_corr,linewidths=0.5, annot=True); #add correrlation inside boxes

In [ ]:
Correlation matrix heatmap: between funding and attendance/crime rate ---add description.

In [ ]:
snd_heat = school_data[['lea_total_expense_num',
'lea_salary_expense_pct',
'lea_services_expense_pct',
'lea_supplies_expense_pct',
'lea_instruct_equip_exp_pct',
'lea_federal_perpupil_num',
'lea_local_perpupil_num',
'lea_state_perpupil_num',            
'lea_avg_daily_attend_pct',
'lea_crime_per_c_num',
'lea_short_susp_per_c_num',
'lea_long_susp_per_c_num',
'lea_expelled_per_c_num',]] 

                       
snd_corr = snd_heat.corr()

plt.figure(figsize = (14,10)) #size of matrix 
sns.heatmap(snd_corr,cmap='RdYlGn_r', linewidths=0.5, annot=True); #add correrlation inside boxes

Correlation matrix heatmap: between race,gender, region ---add description.

In [ ]:
thd_heat = school_data[['SBE District_Northeast',
'SBE District_Northwest',
'SBE District_Piedmont-Triad',
'SBE District_Sandhills',
'SBE District_Southeast',
'SBE District_Southwest',
'SBE District_Western',
'AsianFemalePct',            
'AsianMalePct',
'BlackFemalePct',
'BlackMalePct',
'HispanicFemalePct',
'HispanicMalePct',
'PacificIslandFemalePct',                        
'PacificIslandMalePct',                      
'MinorityFemalePct',
'MinorityMalePct',]] 

                       
thd_corr = thd_heat.corr()

plt.figure(figsize = (14,10)) #size of matrix 
sns.heatmap(thd_corr,linewidths=0.5,annot=True); #add correrlation inside boxes

In [ ]:
df = pd.DataFrame(data = school_data)

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 20))

In [ ]:
df_heat = school_data[['SPG Score', 'MinorityFemalePct','MinorityMalePct',
                       'BlackFemalePct', 'BlackMalePct', 'IndianFemalePct',
                       'AsianFemalePct', 'AsianMalePct' , 'HispanicFemalePct', 'HispanicMalePct', ]]
df_heat_corr = df_heat.corr()

plt.figure(figsize = (16,12))
sns.heatmap(df_heat_corr,annot=True);

# Explore Attributes and Class

Identify and explain interesting relationships between features and the class you are trying to predict (i.e., relationships with variables and the target classification).

In [ ]:
df=pd.DataFrame(school_data)

In [ ]:
def correlation_matrix(school_data):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm
    
      
    fig=plt.figure()
    ax1=fig.add_subplot(111)
    cmap=cm.get_cmap('jet',30)
    cmap = cm.get_cmap('jet', 30)
    cax = ax1.imshow(df.corr(), interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    labels=['column_names',]
    plt.title('Columns Correlations')
    ax1.set_xticklabels(labels,fontsize=3)
    ax1.set_yticklabels(labels,fontsize=3)
    # Add colorbar
    fig.colorbar(cax, ticks=[-1,-.95,-.90,0,.05,.1,.15,.2,.25,.75,.8,.85,.90,.95,1])
    plt.show()

correlation_matrix(df)

In [ ]:
df_heat = school_data[['SPG Score', 'flicensed_teach_pct',#school nad teacher characters
                    'lea_local_perpupil_num','lea_total_expense_num','lea_salary_expense_pct', #funding
                    'lea_supplies_expense_pct','stud_internet_comp_num',#funding
                    'avg_daily_attend_pct','crime_per_c_num', ]] #environment
df_heat_corr = df_heat.corr()

plt.figure(figsize = (16,12))
sns.heatmap(df_heat_corr,annot=True);

In [ ]:
#Created a categorical multiclass variable from SP Grade
SPG_Grade_col = school_data.filter(regex=('^SPG\WGrade')).columns
school_data['SPG Grade']= school_data[SPG_Grade_col].apply(lambda row:'A' if row.any()!=1 else 
                                 row[0]*'A+NG'+row[1]*'B'+row[2]*'C'+row[3]*'D'+row[4]*'F'+row[5]*'I',axis=1)

In [ ]:
expense=school_data[['SPG Score', 'SPG Grade', 'lea_salary_expense_pct', 'lea_services_expense_pct', 'lea_supplies_expense_pct', 'lea_instruct_equip_exp_pct']]

sns.pairplot(expense, hue='SPG Grade');

In [ ]:
minority=school_data[['SPG Score', 'SPG Grade', 'Asian', 'Black', 'Minority', 'Hispanic']]
sns.pairplot(minority, hue='SPG Grade');

# New Features

Are there other features that could be added to the data or created from existing features? Which ones?

In [ ]:
#insert a new feature for SPG as continous values
school_data['SPG']=6
school_data['SPG'] = school_data['SPG Grade_A+NG']*7 + school_data['SPG Grade_B']*5 + school_data['SPG Grade_C']*4 + school_data['SPG Grade_D']*3 + school_data['SPG Grade_F']*2 + school_data['SPG Grade_I']*1

In [ ]:
school_data['Asian']=school_data['AsianFemalePct']+school_data['AsianMalePct']
school_data['Black']=school_data['BlackFemalePct']+school_data['BlackMalePct']
school_data['Hispanic']=school_data['HispanicFemalePct']+school_data['HispanicMalePct']
school_data['Minority']=school_data['MinorityFemalePct']+school_data['MinorityMalePct']

# Exceptional Work

You have free reign to provide additional analyses. One idea: implement dimensionality reduction, then visualize and interpret the results.

PCA - extracting important variables (in form of components) from a large set of variables available in a data set.

when we have so many features, that means that we are working with high dimentional data. therefore we need to reduce our data dimentions. PCA is the Dimentionality reduction techniques that is very usefull in our project. There are 334 features, and only 2443 rows in the data set. Having too many features and limited data makes Principal Component Analysis a great candidate to find a model that predicts the NC school performance grades. We only consider the quantitative columns, and we would like to choose as many components as needed to explain around 80% of the SPG. Based on the variance explained graph, we must choose at least 62 principal components to explain 80% of the SPG. Variance explained graph illustrates the necessity of having 62 components to address 80% of the SPG.

The other dimentionality reduction technique is using Random Forest Classifier.
it lists the top 40 features along with their scores.

In [ ]:
#split data into X and y dataframes

X = school_data.loc[:,school_data.dtypes==float]
y = school_data['SPG']

In [ ]:
#split X and y into test and train sets.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
#applied a scaling procedure to scale the size of variables in the x dataframe

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#run PCA on 62 components of the dataset, which explains 85% of the variance.

from sklearn.decomposition import PCA
pca = PCA(n_components=62)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
d = {'ratio':pca.explained_variance_ratio_,'total':pca.explained_variance_ratio_.cumsum()}

print(d)

In [ ]:
plt.plot(pca.explained_variance_ratio_, '-o')
plt.xlim(0,62)
plt.ylim(0,0.2)
pca.explained_variance_ratio_

In [ ]:
#Run a Kfolds cross validation model on the data set and predicted y from the set

from sklearn.linear_model import LogisticRegressionCV
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
fold = KFold(len(y_train), n_folds=10, shuffle=True)
classifier = LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(-10, 10)))
        ,penalty='l2'
        ,scoring='roc_auc'
        ,cv=fold
        ,max_iter=4000
        ,fit_intercept=True
       ,solver='newton-cg')

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
#Created a confusion matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

In [ ]:
#Can't get this to work
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6])

n_classes = y.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Import `RandomForestClassifier`
from sklearn.ensemble import RandomForestClassifier

# Isolate Data, class labels and column values
X = school_data.iloc[:,0:40]
Y = school_data.iloc[:,-1]
names = school_data.columns.values

# Build the model
rfc = RandomForestClassifier()

# Fit the model
rfc.fit(X, Y)

# Print the results
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rfc.feature_importances_), names), reverse=True))
